# import and load data set

In [1]:
"""
Skeleton + GridSearch on small subset -> train full -> predict
"""
import geopandas as gpd
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split

# from area_classif import best_model
from utils import *

change_type_map = {
    'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
    'Mega Projects': 5
}

# --- Read geojsons
train_df_or = gpd.read_file('train.geojson')
# train_df = train_df.set_crs("OGC:CRS84", allow_override=True)

test_df_or  = gpd.read_file('test.geojson')

In [3]:
train_df = train_df_or.copy()
test_df = test_df_or.copy()

train_y = train_df["change_type"].map(change_type_map).to_numpy()

feature_cols = set()

train_df = encode_one_hot_types(train_df, feature_cols)
test_df  = encode_one_hot_types(test_df, feature_cols)

# train_df = add_geometry_features(train_df, feature_cols)
# test_df  = add_geometry_features(test_df, feature_cols)
train_df = add_max_gap_between_sets(train_df, feature_cols)
test_df = add_max_gap_between_sets(test_df, feature_cols)

train_df = add_last_state(train_df, feature_cols)
test_df  = add_last_state(test_df, feature_cols)

train_df = add_regressed_state(train_df, feature_cols)
test_df  = add_regressed_state(test_df, feature_cols)

train_df = train_df.reindex(columns=feature_cols, fill_value=0)
test_df  = test_df.reindex(columns=feature_cols, fill_value=0)

train_x = train_df.to_numpy(dtype=float)
test_x  = test_df.to_numpy(dtype=float)
print(train_x.shape, train_y.shape, test_x.shape)



(296146, 35) (296146,) (120526, 35)


In [4]:
pipe = Pipeline([
    ("clf", BernoulliNB())  # placeholder
])

param_grid = [
    # ===== BernoulliNB =====
    {
        "clf": [BernoulliNB()],
        "clf__alpha": [0.01, 0.1, 0.5, 1.0],
        "clf__binarize": [None, 0.0],   # None si déjà binaire, 0.0 sinon
        "clf__fit_prior": [True, False],
    },

    # ===== MultinomialNB =====
    {
        "clf": [MultinomialNB()],
        "clf__alpha": [0.01, 0.1, 0.5, 1.0],
        "clf__fit_prior": [True, False],
    }
]


cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

gs = GridSearchCV(
    pipe, param_grid,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1,
    verbose=3,
    refit=True
)

gs.fit(train_x, train_y)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


/home/hp3/Documents/area_classification/.venv/lib/python3.11/site-packages/sklearn/naive_bayes.py:1256: RuntimeWarning: invalid value encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
/home/hp3/Documents/area_classification/.venv/lib/python3.11/site-packages/sklearn/naive_bayes.py:1256: RuntimeWarning: invalid value encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
/home/hp3/Documents/area_classification/.venv/lib/python3.11/site-packages/sklearn/naive_bayes.py:1256: RuntimeWarning: invalid value encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))


[CV 2/3] END clf=BernoulliNB(), clf__alpha=0.01, clf__binarize=None, clf__fit_prior=True;, score=0.032 total time=   1.0s
[CV 2/3] END clf=BernoulliNB(), clf__alpha=0.1, clf__binarize=None, clf__fit_prior=False;, score=0.032 total time=   0.9s
[CV 3/3] END clf=BernoulliNB(), clf__alpha=0.01, clf__binarize=None, clf__fit_prior=True;, score=0.032 total time=   0.9s
[CV 3/3] END clf=BernoulliNB(), clf__alpha=0.01, clf__binarize=0.0, clf__fit_prior=False;, score=0.255 total time=   1.6s


/home/hp3/Documents/area_classification/.venv/lib/python3.11/site-packages/sklearn/naive_bayes.py:1256: RuntimeWarning: invalid value encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))


[CV 3/3] END clf=BernoulliNB(), clf__alpha=0.1, clf__binarize=None, clf__fit_prior=False;, score=0.032 total time=   1.3s


...

[CV 1/3] END clf=BernoulliNB(), clf__alpha=0.1, clf__binarize=0.0, clf__fit_prior=False;, score=0.250 total time=   2.9s
[CV 1/3] END clf=BernoulliNB(), clf__alpha=0.5, clf__binarize=0.0, clf__fit_prior=False;, score=0.249 total time=   2.8s
[CV 2/3] END clf=BernoulliNB(), clf__alpha=1.0, clf__binarize=None, clf__fit_prior=False;, score=0.032 total time=   1.9s
[CV 3/3] END clf=BernoulliNB(), clf__alpha=0.1, clf__binarize=0.0, clf__fit_prior=False;, score=0.254 total time=   3.3s
[CV 3/3] END clf=BernoulliNB(), clf__alpha=0.5, clf__binarize=0.0, clf__fit_prior=True;, score=0.355 total time=   3.0s
[CV 3/3] END clf=BernoulliNB(), clf__alpha=1.0, clf__binarize=None, clf__fit_prior=True;, score=0.032 total time=   2.1s
[CV 1/3] END clf=BernoulliNB(), clf__alpha=1.0, clf__binarize=None, clf__fit_prior=False;, score=0.032 total time=   2.1s
[CV 2/3] END clf=BernoulliNB(), clf__alpha=1.0, clf__binarize=None, clf__fit_prior=True;, score=0.032 total time=   2.3s
[CV 1/3] END clf=BernoulliNB(),

/home/hp3/Documents/area_classification/.venv/lib/python3.11/site-packages/sklearn/naive_bayes.py:1256: RuntimeWarning: invalid value encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))


[CV 2/3] END clf=BernoulliNB(), clf__alpha=0.5, clf__binarize=0.0, clf__fit_prior=True;, score=0.355 total time=   3.4s
[CV 3/3] END clf=BernoulliNB(), clf__alpha=0.5, clf__binarize=0.0, clf__fit_prior=False;, score=0.253 total time=   3.1s
[CV 2/3] END clf=BernoulliNB(), clf__alpha=0.5, clf__binarize=0.0, clf__fit_prior=False;, score=0.256 total time=   3.3s
[CV 3/3] END clf=BernoulliNB(), clf__alpha=1.0, clf__binarize=None, clf__fit_prior=False;, score=0.032 total time=   2.1s
[CV 2/3] END clf=BernoulliNB(), clf__alpha=0.1, clf__binarize=0.0, clf__fit_prior=False;, score=0.257 total time=   4.3s
[CV 2/3] END clf=MultinomialNB(), clf__alpha=0.01, clf__fit_prior=True;, score=nan total time=   1.2s
[CV 2/3] END clf=MultinomialNB(), clf__alpha=0.01, clf__fit_prior=False;, score=nan total time=   1.2s
[CV 1/3] END clf=MultinomialNB(), clf__alpha=0.01, clf__fit_prior=False;, score=nan total time=   1.2s
[CV 3/3] END clf=MultinomialNB(), clf__alpha=0.01, clf__fit_prior=True;, score=nan tota

/home/hp3/Documents/area_classification/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:490: FitFailedWarning: 
24 fits failed out of a total of 72.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "/home/hp3/Documents/area_classification/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 833, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/hp3/Documents/area_classification/.venv/lib/python3.11/site-packages/sklearn/base.py", line 1336, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hp3/Docum

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...rnoulliNB())])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","[{'clf': [BernoulliNB()], 'clf__alpha': [0.01, 0.1, ...], 'clf__binarize': [None, 0.0], 'clf__fit_prior': [True, False]}, {'clf': [MultinomialNB()], 'clf__alpha': [0.01, 0.1, ...], 'clf__fit_prior': [True, False]}]"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'f1_macro'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",StratifiedKFo... shuffle=True)
,"ver

In [10]:
best_model = gs.best_estimator_

print("\n===== BEST MODEL =====")
print(gs.best_estimator_)

print("BEST PARAMS:", gs.best_params_)
print("BEST CV F1:", gs.best_score_)

(30000, 4) (30000,)


In [ ]:
cross_validation(3, best_model, train_x, train_y)

best_model.fit(train_x, train_y)


# --- 4) Predict test
pred_y = best_model.predict(test_x)
print("pred:", pred_y.shape)



In [8]:
# --- 5) Save submission (index = Id = index du test)
pred_df = pd.DataFrame({"change_type": pred_y}, index=test_df.index)
pred_df.index.name = "Id"
pred_df.to_csv("submission.csv")

print("Saved: submission.csv")

Fitting 3 folds for each of 9 candidates, totalling 27 fits
{'svm__C': 100, 'svm__gamma': 'scale'}
